## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler

## Joining Fixture Data to FIFA

In [ ]:
pl = pd.read_csv('/content/premier-league-matches.csv')
pl = pl[pl['Season_End_Year']>=2015]
pl['Date'] = pd.to_datetime(pl['Date'])
pl

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
8606,2015,1,2014-08-16,Manchester Utd,1,2,Swansea City,A
8607,2015,1,2014-08-16,West Brom,2,2,Sunderland,D
8608,2015,1,2014-08-16,Leicester City,2,2,Everton,D
8609,2015,1,2014-08-16,West Ham,0,1,Tottenham,A
8610,2015,1,2014-08-16,QPR,0,1,Hull City,A
...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A


In [ ]:
pl['Home'].unique()

array(['Manchester Utd', 'West Brom', 'Leicester City', 'West Ham', 'QPR',
       'Stoke City', 'Arsenal', 'Liverpool', 'Newcastle Utd', 'Burnley',
       'Aston Villa', 'Swansea City', 'Crystal Palace', 'Southampton',
       'Chelsea', 'Everton', 'Tottenham', 'Hull City', 'Sunderland',
       'Manchester City', 'Norwich City', 'Bournemouth', 'Watford',
       'Middlesbrough', 'Brighton', 'Huddersfield', 'Fulham', 'Wolves',
       'Cardiff City', 'Sheffield Utd', 'Leeds United', 'Brentford',
       "Nott'ham Forest"], dtype=object)

In [ ]:
teams = pd.read_csv('teams.csv')
teams = teams[(teams['league_id'] == 13)]
teams = teams[['team_name','overall','international_prestige', 'domestic_prestige',
       'transfer_budget_eur', 'club_worth_eur','fifa_version', 'fifa_update_date']]
teams['fifa_update_date'] = pd.to_datetime(teams['fifa_update_date'])
teams

<ipython-input-43-b2c53f85dad7>:1: DtypeWarning: Columns (32,38,39,40,41,49,53) have mixed types. Specify dtype option on import or set low_memory=False.
  teams = pd.read_csv('teams.csv')


,team_name,overall,international_prestige,domestic_prestige,transfer_budget_eur,club_worth_eur,fifa_version,fifa_update_date
0,Manchester City,85,10,10.0,NaN,-2.147484e+09,23,2023-01-13
4,Chelsea,84,8,9.0,NaN,-2.147484e+09,23,2023-01-13
5,Liverpool,84,9,9.0,NaN,-2.147484e+09,23,2023-01-13
15,Manchester United,82,8,8.0,NaN,-2.147484e+09,23,2023-01-13
20,Tottenham Hotspur,81,7,8.0,NaN,-2.147484e+09,23,2023-01-13
...,...,...,...,...,...,...,...,...
384511,Sunderland,74,14,12.0,16000000.0,NaN,15,2014-08-29
384541,Crystal Palace,73,11,10.0,9000000.0,NaN,15,2014-08-29
384564,Hull City,72,11,10.0,10000000.0,NaN,15,2014-08-29
384597,Leicester City,70,10,11.0,9500000.0,NaN,15,2014-08-29


In [ ]:
teams['team_name'].unique()

array(['Manchester City', 'Chelsea', 'Liverpool', 'Manchester United',
       'Tottenham Hotspur', 'Arsenal', 'Aston Villa', 'Newcastle United',
       'West Ham United', 'Leicester City', 'Wolverhampton Wanderers',
       'Everton', 'Leeds United', 'Nottingham Forest', 'Fulham',
       'Crystal Palace', 'Brighton & Hove Albion', 'Southampton',
       'Brentford', 'AFC Bournemouth', 'Burnley', 'Watford',
       'Norwich City', 'West Bromwich Albion', 'Sheffield United',
       'Huddersfield Town', 'Cardiff City', 'Stoke City', 'Swansea City',
       'Sunderland', 'Middlesbrough', 'Hull City', 'Queens Park Rangers'],
      dtype=object)

In [ ]:
team_name_mapping = {
    'Manchester Utd': 'Manchester United',
    'West Brom': 'West Bromwich Albion',
    'Leicester City': 'Leicester City',
    'West Ham': 'West Ham United',
    'QPR': 'Queens Park Rangers',
    'Stoke City': 'Stoke City',
    'Arsenal': 'Arsenal',
    'Liverpool': 'Liverpool',
    'Newcastle Utd': 'Newcastle United',
    'Burnley': 'Burnley',
    'Aston Villa': 'Aston Villa',
    'Swansea City': 'Swansea City',
    'Crystal Palace': 'Crystal Palace',
    'Southampton': 'Southampton',
    'Chelsea': 'Chelsea',
    'Everton': 'Everton',
    'Tottenham': 'Tottenham Hotspur',
    'Hull City': 'Hull City',
    'Sunderland': 'Sunderland',
    'Manchester City': 'Manchester City',
    'Norwich City': 'Norwich City',
    'Bournemouth': 'AFC Bournemouth',
    'Watford': 'Watford',
    'Middlesbrough': 'Middlesbrough',
    'Brighton': 'Brighton & Hove Albion',
    'Huddersfield': 'Huddersfield Town',
    'Fulham': 'Fulham',
    'Wolves': 'Wolverhampton Wanderers',
    'Cardiff City': 'Cardiff City',
    'Sheffield Utd': 'Sheffield United',
    'Leeds United': 'Leeds United',
    'Brentford': 'Brentford',
    "Nott'ham Forest": 'Nottingham Forest'
}

In [ ]:
pl['Home'] = pl['Home'].map(team_name_mapping).fillna(pl['Home'])
pl['Away'] = pl['Away'].map(team_name_mapping).fillna(pl['Away'])

In [ ]:
def find_closest_date(fixture_date, team, ratings):
    team_data = ratings[ratings['team_name'] == team]
    team_data = team_data.reset_index(drop=True)
    closest_row = team_data.iloc[(team_data['fifa_update_date'] - fixture_date).abs().idxmin()]
    return closest_row[1:]

In [ ]:
home_data = pl.apply(lambda row: find_closest_date(row['Date'], row['Home'], teams), axis=1)
home_data.columns = ['home_' + col for col in home_data.columns]
away_data = pl.apply(lambda row: find_closest_date(row['Date'], row['Away'], teams),axis=1)
away_data.columns = ['away_' + col for col in away_data.columns]
fixtures = pd.concat([pl, home_data, away_data], axis=1)
fixtures

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,home_overall,home_international_prestige,...,home_club_worth_eur,home_fifa_version,home_fifa_update_date,away_overall,away_international_prestige,away_domestic_prestige,away_transfer_budget_eur,away_club_worth_eur,away_fifa_version,away_fifa_update_date
8606,2015,1,2014-08-16,Manchester United,1,2,Swansea City,A,81,20,...,NaN,15,2014-08-29,75,14,10.0,9500000.0,NaN,15,2014-08-29
8607,2015,1,2014-08-16,West Bromwich Albion,2,2,Sunderland,D,75,14,...,NaN,15,2014-08-29,74,14,12.0,16000000.0,NaN,15,2014-08-29
8608,2015,1,2014-08-16,Leicester City,2,2,Everton,D,70,10,...,NaN,15,2014-08-29,78,16,14.0,14000000.0,NaN,15,2014-08-29
8609,2015,1,2014-08-16,West Ham United,0,1,Tottenham Hotspur,A,74,14,...,NaN,15,2014-08-29,79,17,17.0,26000000.0,NaN,15,2014-08-29
8610,2015,1,2014-08-16,Queens Park Rangers,0,1,Hull City,A,75,14,...,NaN,15,2014-08-29,72,11,10.0,10000000.0,NaN,15,2014-08-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,AFC Bournemouth,H,77,4,...,896600000.0,23,2023-01-13,74,1,2.0,NaN,2.000000e+08,23,2023-01-13
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham United,H,79,6,...,882300000.0,23,2023-01-13,79,5,6.0,NaN,8.585000e+08,23,2023-01-13
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton & Hove Albion,H,79,3,...,715400000.0,23,2023-01-13,76,2,3.0,NaN,3.550000e+08,23,2023-01-13
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham Hotspur,A,76,2,...,763100000.0,23,2023-01-13,81,7,8.0,NaN,-2.147484e+09,23,2023-01-13


In [ ]:
fixtures.to_csv('full_data_sample.csv', index=False)

## Updating League Table

In [ ]:
fdf = pd.read_csv('/content/merged-fixtures-draft.csv')
fdf.columns

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR', 'home_overall', 'home_international_prestige',
       'home_domestic_prestige', 'home_transfer_budget_eur',
       'home_manager_change', 'home_interim', 'away_overall',
       'away_international_prestige', 'away_domestic_prestige',
       'away_transfer_budget_eur', 'away_manager_change', 'away_interim',
       'fifa_update_date'],
      dtype='object')

In [ ]:
premier_league_2014 = {
    "Manchester City": 1,
    "Liverpool": 2,
    "Chelsea": 3,
    "Arsenal": 4,
    "Everton": 5,
    "Tottenham Hotspur": 6,
    "Manchester United": 7,
    "Southampton": 8,
    "Newcastle United": 10,
    "Stoke City": 9,
    "Aston Villa": 15,
    "Hull City": 16,
    "Crystal Palace": 11,
    "Swansea City": 12,
    "West Bromwich Albion": 17,
    "Sunderland": 14,
    "Norwich City": 18,
    "Fulham": 19,
    "Cardiff City": 20,
    "West Ham United": 13
}

In [ ]:
def update_league_positions(df):
    df['homepos'] = 0
    df['awaypos'] = 0
    df['home_prev_season_pos'] = 0
    df['away_prev_season_pos'] = 0
    league_table = {}
    last_updated_date = None
    prev_season_table = {}

    for index, row in df.iterrows():
        season = row['Season_End_Year']
        home_team = row['Home']
        away_team = row['Away']
        home_goals = row['HomeGoals']
        away_goals = row['AwayGoals']
        current_date = row['Date']
        gameweek = row['Wk']

        if index > 0 and df.loc[index - 1, 'Season_End_Year'] != season:
            prev_season_table = {team: pos for pos, (team, _) in enumerate(sorted(league_table.items(),
                                key=lambda x: (-x[1]['points'], -x[1]['goal_diff'], -x[1]['goals_scored'])),start=1)}
            league_table = {}


        for team in [home_team, away_team]:
            if team not in league_table:
                league_table[team] = {'points': 0, 'goal_diff': 0, 'goals_scored': 0, 'games_played': 0}

        if last_updated_date != current_date:
            ranked_teams = sorted(league_table.items(),
                key=lambda x: (-x[1]['points'], -x[1]['goal_diff'], -x[1]['goals_scored']))
            rankings = {team: rank + 1 for rank, (team, _) in enumerate(ranked_teams)}
            last_updated_date = current_date

        if home_goals > away_goals:
            league_table[home_team]['points'] += 3
        elif home_goals < away_goals:
            league_table[away_team]['points'] += 3
        else:
            league_table[home_team]['points'] += 1
            league_table[away_team]['points'] += 1

        league_table[home_team]['goal_diff'] += (home_goals - away_goals)
        league_table[away_team]['goal_diff'] += (away_goals - home_goals)
        league_table[home_team]['goals_scored'] += home_goals
        league_table[away_team]['goals_scored'] += away_goals
        league_table[home_team]['games_played'] += 1
        league_table[away_team]['games_played'] += 1

        if home_team in prev_season_table:
            df.at[index, 'home_prev_season_pos'] = prev_season_table[home_team]
        elif season == 2015 and home_team in premier_league_2014:
            df.at[index, 'home_prev_season_pos'] = premier_league_2014[home_team]
        else:
            df.at[index, 'home_prev_season_pos'] = 22
        if away_team in prev_season_table:
            df.at[index, 'away_prev_season_pos'] = prev_season_table[away_team]
        elif season == 2015 and away_team in premier_league_2014:
            df.at[index, 'away_prev_season_pos'] = premier_league_2014[away_team]
        else:
            df.at[index, 'away_prev_season_pos'] = 22

        if (home_team in rankings) and (gameweek != 1):
            df.at[index, 'homepos'] = rankings[home_team]
        else:
            df.at[index, 'homepos'] = 0

        if (away_team in rankings) and (gameweek != 1):
            df.at[index, 'awaypos'] = rankings[away_team]
        else:
            df.at[index, 'awaypos'] = 0

    return df

In [ ]:
updated_fixture_list = update_league_positions(fdf)

In [ ]:
view_test = updated_fixture_list[['Wk', 'Date', 'Home', 'HomeGoals', 'Away', 'AwayGoals', 'homepos', 'awaypos', 'home_prev_season_pos', 'away_prev_season_pos']]
view_test

,Wk,Date,Home,HomeGoals,Away,AwayGoals,homepos,awaypos,home_prev_season_pos,away_prev_season_pos
0,1,16/08/2014,Manchester United,1,Swansea City,2,0,0,7,12
1,1,16/08/2014,West Bromwich Albion,2,Sunderland,2,0,0,17,14
2,1,16/08/2014,Leicester City,2,Everton,2,0,0,22,5
3,1,16/08/2014,West Ham United,0,Tottenham Hotspur,1,0,0,13,6
4,1,16/08/2014,Queens Park Rangers,0,Hull City,1,0,0,22,16
...,...,...,...,...,...,...,...,...,...,...
3415,38,28/05/2023,Everton,1,AFC Bournemouth,0,17,15,16,22
3416,38,28/05/2023,Leicester City,2,West Ham United,1,18,14,8,7
3417,38,28/05/2023,Aston Villa,2,Brighton & Hove Albion,1,7,6,14,9
3418,38,28/05/2023,Leeds United,1,Tottenham Hotspur,4,19,8,17,4


In [ ]:
updated_fixture_list = updated_fixture_list.fillna(0)

In [ ]:
updated_fixture_list.to_csv('fixtures_data_epl_2015-2023.csv', index=False)

## 9-game team windows

In [ ]:
fixtures_data = pd.read_csv('/content/fixtures_data_epl_2015-2023.csv')
fixtures_data['Date'] = pd.to_datetime(fixtures_data['Date'], format='%d/%m/%Y')
fixtures_data['fifa_update_date'] = pd.to_datetime(fixtures_data['fifa_update_date'], format='%d/%m/%Y')

In [ ]:
fixtures_data.columns

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR', 'home_overall', 'home_international_prestige',
       'home_domestic_prestige', 'home_transfer_budget_eur',
       'home_manager_change', 'home_interim', 'away_overall',
       'away_international_prestige', 'away_domestic_prestige',
       'away_transfer_budget_eur', 'away_manager_change', 'away_interim',
       'fifa_update_date', 'homepos', 'awaypos', 'home_prev_season_pos',
       'away_prev_season_pos'],
      dtype='object')

In [ ]:
def calculate_points(row, team_type):
    if row['FTR'] == 'H' and team_type == 'home':
        return 3
    elif row['FTR'] == 'A' and team_type == 'away':
        return 3
    elif row['FTR'] == 'D':
        return 1
    return 0

def aggregate_window(window, team):
        return {'goals_scored': window.apply(lambda x: x['HomeGoals'] if x['Home'] == team else x['AwayGoals'], axis=1).sum(),
            'goals_conceded': window.apply(lambda x: x['AwayGoals'] if x['Home'] == team else x['HomeGoals'], axis=1).sum(),
            'points': window.apply(lambda x: x['home_points'] if x['Home'] == team else x['away_points'], axis=1).sum(),
            'home_games': (window['Home'] == team).sum(),
            'rating': window.apply(lambda x: x['home_overall'] if x['Home'] == team else x['away_overall'], axis=1).mean(),
            'opponent_rating': window.apply(lambda x: x['away_overall'] if x['Home'] == team else x['home_overall'], axis=1).mean(),
            'avg_rating_gap':  (window.apply(lambda x: x['home_overall'] if x['Home'] == team else x['away_overall'], axis=1).mean()) - (window.apply(lambda x: x['away_overall'] if x['Home'] == team else x['home_overall'], axis=1).mean()),
            'total_prestige':   window.apply(lambda x: (x['home_domestic_prestige'] + x['home_international_prestige'])/2 if x['Home'] == team else (x['away_domestic_prestige'] + x['away_international_prestige'])/2, axis=1).sum(),
            'avg_budget': window.apply(lambda x: x['home_transfer_budget_eur'] if x['Home'] == team else x['away_transfer_budget_eur'], axis=1).mean(),
            'avg_league_position': window.apply(lambda x: x['homepos'] if x['Home'] == team else x['awaypos'], axis=1).mean(),
            'prev_season_position': window.apply(lambda x: x['home_prev_season_pos'] if x['Home'] == team else x['away_prev_season_pos'], axis=1).mean()}

def treatment_features(team, date, fixtures, window=9):
    pre_window = fixtures[
        (fixtures['Date'] < date) & ((fixtures['Home'] == team) | (fixtures['Away'] == team))
    ].sort_values('Date', ascending=False).head(window)

    post_window = fixtures[
        (fixtures['Date'] > date) & ((fixtures['Home'] == team) | (fixtures['Away'] == team))
    ].sort_values('Date', ascending=True).head(window)

    return {'pre_treatment': aggregate_window(pre_window, team),
        'post_treatment': aggregate_window(post_window, team ), }

def identify_gameweek_boundaries(fixtures_data, lower_gameweek=9, upper_gameweek=30):
    """Identify the date boundaries for valid gameweeks."""
    season_boundaries = {}
    for season, season_data in fixtures_data.groupby('Season_End_Year'):
        season_start = season_data['Date'].min()
        season_data = season_data.copy()
        lower_bound_date = season_data.loc[season_data['Wk'] == lower_gameweek, 'Date'].iloc[0]
        upper_bound_date = season_data.loc[season_data['Wk'] == upper_gameweek, 'Date'].iloc[0]
        season_boundaries[season] = {'lower_date': lower_bound_date, 'upper_date': upper_bound_date}
    return season_boundaries

def within_gameweek_boundaries(row, boundaries):
    season = row['Season_End_Year']
    lower_date = boundaries[season]['lower_date']
    upper_date = boundaries[season]['upper_date']
    return lower_date <= row['Date'] <= upper_date


In [ ]:
fixtures_data['home_points'] = fixtures_data.apply(calculate_points, team_type='home', axis=1)
fixtures_data['away_points'] = fixtures_data.apply(calculate_points, team_type='away', axis=1)

In [ ]:
boundaries = identify_gameweek_boundaries(fixtures_data)

In [ ]:
treated_fixtures = fixtures_data[(fixtures_data['home_manager_change'] == 1) | (fixtures_data['away_manager_change'] == 1)]
treated_fixtures = treated_fixtures[treated_fixtures.apply(within_gameweek_boundaries, boundaries=boundaries, axis=1)]

treated_features = []
for _, row in treated_fixtures.iterrows():
    team = row['Home'] if row['home_manager_change'] == 1 else row['Away']
    season = row['Season_End_Year']
    features = treatment_features(team, row['Date'], fixtures_data)
    features.update({'team': team,'season': season, 'treatment': 1, 'manager_change_date': row['Date']})
    treated_features.append(features)

treated_features_df = pd.DataFrame(treated_features)

In [ ]:
treated_features_df

,pre_treatment,post_treatment,team,season,treatment,manager_change_date
0,"{'goals_scored': 7, 'goals_conceded': 14, 'poi...","{'goals_scored': 11, 'goals_conceded': 9, 'poi...",Crystal Palace,2015,1,2014-12-28
1,"{'goals_scored': 5, 'goals_conceded': 15, 'poi...","{'goals_scored': 8, 'goals_conceded': 7, 'poin...",West Bromwich Albion,2015,1,2015-01-01
2,"{'goals_scored': 6, 'goals_conceded': 10, 'poi...","{'goals_scored': 14, 'goals_conceded': 10, 'po...",Crystal Palace,2015,1,2015-01-10
3,"{'goals_scored': 10, 'goals_conceded': 15, 'po...","{'goals_scored': 14, 'goals_conceded': 16, 'po...",Queens Park Rangers,2015,1,2015-02-07
4,"{'goals_scored': 2, 'goals_conceded': 14, 'poi...","{'goals_scored': 16, 'goals_conceded': 14, 'po...",Aston Villa,2015,1,2015-02-21
...,...,...,...,...,...,...
57,"{'goals_scored': 7, 'goals_conceded': 11, 'poi...","{'goals_scored': 16, 'goals_conceded': 11, 'po...",Aston Villa,2023,1,2022-10-20
58,"{'goals_scored': 13, 'goals_conceded': 16, 'po...","{'goals_scored': 11, 'goals_conceded': 18, 'po...",Leeds United,2023,1,2023-02-08
59,"{'goals_scored': 4, 'goals_conceded': 8, 'poin...","{'goals_scored': 17, 'goals_conceded': 10, 'po...",Crystal Palace,2023,1,2023-03-19
60,"{'goals_scored': 13, 'goals_conceded': 16, 'po...","{'goals_scored': 11, 'goals_conceded': 17, 'po...",Leicester City,2023,1,2023-04-04


In [ ]:
control_fixtures = fixtures_data[(fixtures_data['home_manager_change'] == 0) | (fixtures_data['away_manager_change'] == 0)]
control_fixtures = control_fixtures[fixtures_data.apply(within_gameweek_boundaries, boundaries=boundaries, axis=1)]
control_features = []
for team, team_fixtures in control_fixtures.groupby(['Home', 'Season_End_Year']):
    random_date = team_fixtures['Date'].sample(1).iloc[0]
    features = treatment_features(team[0], random_date, fixtures_data)
    features.update({'team': team[0], 'season': team[1], 'treatment': 0})
    control_features.append(features)

<ipython-input-48-9a81f90348f9>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_fixtures = control_fixtures[fixtures_data.apply(within_gameweek_boundaries, boundaries=boundaries, axis=1)]


In [ ]:
control_features_df = pd.DataFrame(control_features)
control_features_df

,pre_treatment,post_treatment,team,season,treatment
0,"{'goals_scored': 13, 'goals_conceded': 11, 'po...","{'goals_scored': 8, 'goals_conceded': 23, 'poi...",AFC Bournemouth,2016,0
1,"{'goals_scored': 14, 'goals_conceded': 21, 'po...","{'goals_scored': 14, 'goals_conceded': 12, 'po...",AFC Bournemouth,2017,0
2,"{'goals_scored': 11, 'goals_conceded': 17, 'po...","{'goals_scored': 15, 'goals_conceded': 16, 'po...",AFC Bournemouth,2018,0
3,"{'goals_scored': 18, 'goals_conceded': 13, 'po...","{'goals_scored': 11, 'goals_conceded': 16, 'po...",AFC Bournemouth,2019,0
4,"{'goals_scored': 2, 'goals_conceded': 16, 'poi...","{'goals_scored': 9, 'goals_conceded': 22, 'poi...",AFC Bournemouth,2020,0
...,...,...,...,...,...
175,"{'goals_scored': 16, 'goals_conceded': 13, 'po...","{'goals_scored': 10, 'goals_conceded': 10, 'po...",Wolverhampton Wanderers,2019,0
176,"{'goals_scored': 14, 'goals_conceded': 11, 'po...","{'goals_scored': 15, 'goals_conceded': 13, 'po...",Wolverhampton Wanderers,2020,0
177,"{'goals_scored': 9, 'goals_conceded': 12, 'poi...","{'goals_scored': 10, 'goals_conceded': 12, 'po...",Wolverhampton Wanderers,2021,0
178,"{'goals_scored': 10, 'goals_conceded': 9, 'poi...","{'goals_scored': 9, 'goals_conceded': 5, 'poin...",Wolverhampton Wanderers,2022,0


In [ ]:
final_features_df = pd.concat([treated_features_df, control_features_df], ignore_index=True)
pre_df = pd.json_normalize(final_features_df['pre_treatment']).add_prefix('pre_')
post_df = pd.json_normalize(final_features_df['post_treatment']).add_prefix('post_')
full_df = pd.concat([final_features_df.drop(columns=['pre_treatment', 'post_treatment']), pre_df, post_df], axis=1)

In [ ]:
full_df.columns

Index(['team', 'season', 'treatment', 'manager_change_date',
       'pre_goals_scored', 'pre_goals_conceded', 'pre_points',
       'pre_home_games', 'pre_rating', 'pre_opponent_rating',
       'pre_avg_rating_gap', 'pre_total_prestige', 'pre_avg_budget',
       'pre_avg_league_position', 'pre_prev_season_position',
       'post_goals_scored', 'post_goals_conceded', 'post_points',
       'post_home_games', 'post_rating', 'post_opponent_rating',
       'post_avg_rating_gap', 'post_total_prestige', 'post_avg_budget',
       'post_avg_league_position', 'post_prev_season_position'],
      dtype='object')

In [ ]:
scaler = MinMaxScaler(feature_range=(1, 20))
full_df['scaled_avg_budget'] = scaler.fit_transform(full_df['pre_avg_budget'].values.reshape(-1, 1))

In [ ]:
full_df = full_df.drop(columns=['post_total_prestige', 'post_avg_budget', 'post_prev_season_position'])
full_df = full_df.rename(columns={'pre_total_prestige': 'total_prestige', 'pre_avg_budget': 'avg_budget', 'pre_prev_season_position': 'prev_season_position'})
full_df['pre_score'] = (0.3*full_df['pre_goals_scored'] - 0.2*full_df['pre_goals_conceded'] +0.25*full_df['pre_points'] +0.25*full_df['pre_avg_league_position']+0.2*(full_df['pre_opponent_rating']/5)+0.1*(20-full_df['pre_rating']/5)+0.1*2*(10-full_df['pre_home_games']))
full_df['post_score'] = (0.3*full_df['post_goals_scored'] - 0.2*full_df['post_goals_conceded'] +0.25*full_df['post_points'] +0.25*full_df['post_avg_league_position']+0.2*(full_df['post_opponent_rating']/4)+0.1*(20-full_df['post_rating']/5)+0.1*2*(10-full_df['post_home_games']))
full_df['expectation'] = 0.3*full_df['total_prestige'] + 0.3*full_df['scaled_avg_budget'] + 0.3*full_df['prev_season_position'] + 0.1*(full_df['pre_rating']/5)

In [ ]:
full_df

,team,season,treatment,manager_change_date,pre_goals_scored,pre_goals_conceded,pre_points,pre_home_games,pre_rating,pre_opponent_rating,...,post_points,post_home_games,post_rating,post_opponent_rating,post_avg_rating_gap,post_avg_league_position,scaled_avg_budget,pre_score,post_score,expectation
0,AFC Bournemouth,2016,0,NaT,13,11,14,5,73.000000,76.555556,...,4,5,73.000000,79.333333,-6.333333,12.222222,2.717514,14.135556,7.497222,30.475254
1,AFC Bournemouth,2017,0,NaT,14,22,8,5,74.000000,78.777778,...,12,4,74.000000,79.111111,-5.111111,13.888889,4.585311,9.721111,13.677778,15.755593
2,AFC Bournemouth,2018,0,NaT,13,17,12,4,75.000000,80.444444,...,7,5,76.000000,78.111111,-2.111111,10.777778,5.433333,12.778889,9.050000,13.930000
3,AFC Bournemouth,2019,0,NaT,18,13,19,4,76.000000,77.888889,...,9,5,76.000000,79.000000,-3.000000,7.666667,5.705273,14.345556,9.816667,14.831582
4,AFC Bournemouth,2020,0,NaT,7,11,6,4,77.000000,78.666667,...,10,5,76.555556,77.222222,-0.666667,16.555556,4.918079,9.040000,11.386111,12.615424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Wolverhampton Wanderers,2019,0,NaT,14,14,16,5,76.000000,79.666667,...,12,3,76.444444,77.666667,-1.222222,7.333333,7.440678,13.177778,11.805556,15.752203
189,Wolverhampton Wanderers,2020,0,NaT,14,8,17,4,78.000000,77.777778,...,12,4,77.888889,79.888889,-2.000000,7.555556,5.508475,14.712222,11.736111,14.762542
190,Wolverhampton Wanderers,2021,0,NaT,8,11,13,4,78.000000,78.000000,...,8,4,78.888889,79.777778,-0.888889,11.222222,5.758945,10.987778,10.322222,16.937684
191,Wolverhampton Wanderers,2022,0,NaT,5,6,12,4,78.000000,78.333333,...,15,4,78.000000,78.555556,-0.555556,8.111111,5.830508,10.656667,12.955556,19.359153


In [ ]:
grouped = full_df.groupby(['team', 'season'])
def filter_treatment(group):
    if len(group['treatment'].unique()) == 2:
        return group[group['treatment'] != 0]
    else:
        return group

full_df_removed = grouped.apply(filter_treatment).reset_index(drop=True)

<ipython-input-53-53b857d361a0>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df_removed = grouped.apply(filter_treatment).reset_index(drop=True)


In [ ]:
full_df_removed

,team,season,treatment,manager_change_date,pre_goals_scored,pre_goals_conceded,pre_points,pre_home_games,pre_rating,pre_opponent_rating,...,post_points,post_home_games,post_rating,post_opponent_rating,post_avg_rating_gap,post_avg_league_position,scaled_avg_budget,pre_score,post_score,expectation
0,AFC Bournemouth,2016,0,NaT,13,11,14,5,73.000000,76.555556,...,4,5,73.000000,79.333333,-6.333333,12.222222,2.717514,14.135556,7.497222,30.475254
1,AFC Bournemouth,2017,0,NaT,14,21,6,4,74.000000,79.000000,...,15,5,74.000000,78.111111,-4.111111,13.111111,4.585311,9.907778,14.383333,15.755593
2,AFC Bournemouth,2018,0,NaT,11,17,7,5,75.000000,78.222222,...,13,4,75.444444,78.222222,-2.777778,12.222222,5.433333,10.612222,13.330556,13.930000
3,AFC Bournemouth,2019,0,NaT,18,13,19,4,76.000000,77.888889,...,9,5,76.000000,79.000000,-3.000000,7.666667,5.705273,14.345556,9.816667,14.831582
4,AFC Bournemouth,2020,0,NaT,2,16,4,4,76.777778,78.222222,...,4,4,76.000000,78.444444,-2.444444,17.444444,4.918079,7.526667,9.383333,12.610979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Wolverhampton Wanderers,2019,0,NaT,16,13,16,5,76.000000,79.555556,...,12,4,76.666667,78.000000,-1.333333,7.777778,7.440678,13.667778,11.627778,15.752203
189,Wolverhampton Wanderers,2020,0,NaT,14,11,16,4,78.000000,78.666667,...,14,4,78.000000,79.000000,-1.000000,7.333333,5.508475,14.870000,12.933333,14.762542
190,Wolverhampton Wanderers,2021,0,NaT,9,12,11,5,78.444444,78.777778,...,9,4,79.000000,78.444444,0.555556,13.222222,5.830508,10.687778,11.802778,17.568041
191,Wolverhampton Wanderers,2022,0,NaT,10,9,17,4,78.111111,76.333333,...,16,4,78.000000,80.222222,-2.222222,8.444444,5.830508,13.363333,13.572222,19.361375


In [ ]:
full_df_removed.to_csv('full_data_epl_2015-2023.csv', index=False)